In [ ]:
import matplotlib.pyplot as plt

import lsst.afw.cameraGeom.utils as cameraGeomUtils
import lsst.afw.display as afwDisplay
import lsst.daf.persistence as dafPersist

%matplotlib inline 
%config InlineBackend.figure_format = 'retina'

afwDisplay.setDefaultBackend("matplotlib")

In [ ]:
dataPath = "/project/shared/auxTel"
butler = dafPersist.Butler(dataPath)

In [ ]:
dataId = dict(dayObs='2020-03-14', seqnum=323)
raw = butler.get('raw', dataId)
output_filename = "M104.fits"

In [ ]:
disp = afwDisplay.Display(1)    # you only need this in every cell because I'm using %matplotlib notebook. DM Bug??
disp.scale('asinh', 'zscale')
disp.mtv(raw, title=dataId.items())

In [ ]:
from lsst.ip.isr import AssembleCcdTask

config = AssembleCcdTask.ConfigClass()          # Configuration options
config.doTrim = True
assembleTask = AssembleCcdTask(config=config)   # Build the Task object
exposure = assembleTask.assembleCcd(raw)        # Do the work.

In [ ]:
disp = afwDisplay.Display(1)
disp.scale('asinh', 'zscale')
disp.mtv(exposure, title=dataId.items())

In [ ]:
bias = butler.get('bias', dataId)
dark = butler.get('dark', dataId)
#linearizer = butler.get('linearizer', dataId)
#defects = butler.get('defects', dataId)
#flat = butler.get('flat', dataId)

In [ ]:
from lsst.ip.isr import IsrTask
isrConfig = IsrTask.ConfigClass()
isrConfig.doDark = False
isrConfig.doWrite = False
isrConfig.doSaturation = False
isrConfig.doSaturationInterpolation = False
isrConfig.overscanNumLeadingColumnsToSkip = 20
isrConfig.doLinearize = False
isrConfig.doDefect = False
isrConfig.doFlat = False
isrTask = IsrTask(config=isrConfig)
postIsr = isrTask.run(raw, bias=bias, dark=dark).exposure

In [ ]:
disp = afwDisplay.Display(1)
disp.scale('asinh', 'zscale')
disp.mtv(postIsr, title=dataId.items())

In [ ]:
postIsr.writeFits(output_filename)